In [ ]:
# |default_exp core
# This will create a package named ssi_analysis_utility/core.py

In [ ]:
# |hide
import nbdev
from nbdev.showdoc import *  # ignore this Pylance warning in favor of following nbdev docs

For help with the Markdown language, see [this guide](https://www.markdownguide.org/basic-syntax/).

# Global static vars
These are used to modify the template for individual use cases

In [ ]:
# |export
# Need the ssi_analysis_utility for a few functions, this can be considered a static var

import importlib
import importlib.util
import os

PACKAGE_NAME: str = (
    "ssi_analysis_utility"  # Make sure to adjust this to your package name
)
DEV_MODE: bool = (
    False  # set below to override, as this is in an export block it'll be exported while the dev mode section is not
)

PACKAGE_DIR = None
try:
    spec = importlib.util.find_spec(PACKAGE_NAME)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    PACKAGE_DIR = os.path.dirname(module.__file__)
except ImportError:
    DEV_MODE = True
except AttributeError:
    DEV_MODE = True
PROJECT_DIR = os.getcwd()  # override value in dev mode
if PROJECT_DIR.endswith('nbs'):
    DEV_MODE=True
    PROJECT_DIR=os.path.split(PROJECT_DIR)[0]

# Dev mode
If you're developing this versus running this, you'll have access to slightly different things. Notable the nbdev functions are only for development and not for runtime. This matters for items such as the config. So we need to detect if you are in dev mode or not and the code has to adjust accordingly. Notice that this section is not exported so will only work in the notebook and not in the module.

In [ ]:
# This section uses nbdev functions so should not be exported as it's for dev purposes
import os

if DEV_MODE:
    PACKAGE_DIR = nbdev.config.get_config(cfg_name="settings.ini", path=os.getcwd())[
        "lib_path"
    ]  # the library is the package of course
    PROJECT_DIR = nbdev.config.get_config(
        cfg_name="settings.ini", path=os.getcwd()
    ).config_path  # the default location of nbdev config file (settings.ini)

# Core

 A module which contains common functions to be used by other modules. Those that exist in the template are meant to be common functions we can use against multiple packages.

#|hide

Notebook blocks starting with #|hide are not shown in the documentation and not exported to the python package. Blocks with #|export are exported to the python package. Blocks with neither are shown to the documentation but not exported to the python package.

## Libraries

Currently all libraries included are listed at the top and calls to them are also made in the block of code that uses them. This is for readability and the performance hit of the import is negligible.

In [ ]:
# |export
# standard libs
import os
import re

# Common to template
# add into settings.ini, requirements, package name is python-dotenv, for conda build ensure `conda config --add channels conda-forge`
import dotenv  # for loading config from .env files, https://pypi.org/project/python-dotenv/
import envyaml  # Allows to loads env vars into a yaml file, https://github.com/thesimj/envyaml
import fastcore  # To add functionality related to nbdev development, https://github.com/fastai/fastcore/
import pandas  # For sample sheet manipulation
from fastcore import (
    test,
)
from fastcore.script import (
    call_parse,
)  # for @call_parse, https://fastcore.fast.ai/script

# Project specific libraries

## Config

Our config file holds all program and user specific variables. This is a good practice to follow as it allows us to easily change variables without having to change code. It also allows us to easily change variables based on the environment we are running in. For example, we may want to run a program in a test environment with a different database than we would in production. This is also a good practice to follow as it allows us to easily change variables without having to change code. It also allows us to easily change variables based on the environment we are running in. For example, we may want to run a program in a test environment with a different database than we would in production.

Configuration is templated to rely on environment (ENV) variables. A default ENV config is provided in `./config/config.default.env` and more advanced data structures are supported in `./config/config.default.yaml`. The `.yaml` file is meant to represent what your program actually works with and the `.env` file options the user can change at run time.

Make sure you know the priority of variables and check on them when debugging your code. Also ensure that your yaml file is referenced appropriately in the `.env` file. 

When in use there's an expectation you'll have multiple config files for different use cases e.g. development, production environment for different paths, etc.

### set env variables
A helper function for getting your config values, this will set the environment variables with the provided `.env` values. If you're missing values it'll ensure they're loaded in with the defaults file.

In [ ]:
# |export
import importlib
import importlib.util


def set_env_variables(config_path: str, overide_env_vars: bool = True) -> bool:
    # Load dot env sets environmental values from a file, if the value already exists it will not be overwritten unless override is set to True.
    # If we have multiple .env files then we need to apply the one which we want to take precedence last with overide.

    # Order of precedence: .env file > environment variables > default values
    # When developing, making a change to the config will not be reflected until the environment is restarted

    # Set the env vars first, this is needed for the card.yaml to replace ENV variables
    # NOTE: You need to adjust PROJECT_NAME to your package name for this to work, the exception is only for dev purposes
    # This here checks if your package is installed, such as through pypi or through pip install -e  [.dev] for development. If it is then it'll go there and use the config files there as your default values.
    try:
        dotenv.load_dotenv(
            f"{PACKAGE_DIR}/config/config.default.env", override=False
        )
    except Exception as e:
        print(f"Error: {PACKAGE_DIR}/config/config.default.env does not exist")
        return False

    # 2. set values from file:
    if os.path.isfile(config_path):
        dotenv.load_dotenv(config_path, override=overide_env_vars)

    return True

### get config

When you run this function, assuming things are set up properly, you end up with a dict that matches your `.yaml` file. This file will have all the inputs for the package and settings of your program.

To do this it will use a `.env` config file, which has an associated yaml file defined with `CORE_YAML_CONFIG_FILE` in the `.env` file. And then use the `.env` file to load values into the associated `.yaml` file.

In [ ]:
# |export
import importlib
import importlib.util


def get_config(config_path: str = None, overide_env_vars: bool = True) -> dict:
    if config_path is None:
        config_path = ""
    # First sets environment with variables from config_path, then uses those variables to fill in appropriate values in the config.yaml file, the yaml file is then returned as a dict
    # If you want user env variables to take precedence over the config.yaml file then set overide_env_vars to False
    set_env_variables(config_path, overide_env_vars)
   
    config: dict = envyaml.EnvYAML(
        os.environ.get(
            "CORE_YAML_CONFIG_FILE", f"{PACKAGE_DIR}/config/config.default.yaml"
        ),
        strict=False,
    ).export()

    return config

### Variables

All the user input variables and machine adjustable variables should be in your config, which is a dict. Reference config.default.yaml for how to access your variables. Also note that with python dicts you can use `dict_variable.get("variable", default_value)` to ensure that you don't get a key error if the variable is not set.

In [ ]:
# |export
# create a os.PathLike object
config = get_config(os.environ.get("CORE_CONFIG_FILE", ""))

### show project env vars
A helper function intended to only be used with debugging. It shows all your project specific environmental variables.

In [ ]:
# |export
def show_project_env_vars(config: dict) -> None:
    # Prints out all the project environment variables
    # This is useful for debugging and seeing what is being set
    for k, v in config.items():
        # If ENV var starts with PROJECTNAME_ then print
        if k.startswith(config["CORE_PROJECT_VARIABLE_PREFIX"]):
            print(f"{k}={v}")

In [ ]:
# |hide
show_project_env_vars(config)

## get_samplesheet
This function is to unify the way we work with sample_sheet's which is for us a file with a table of values, typically samples for batch processing. We want to approach doing it this way so all programs have batch processing in mind and working with the same data structure.

To make use of it we have a small sample_sheet yaml object which looks like
    
```yaml
sample_sheet:
    path: path/to/sample_sheet.tsv
    delimiter: '\t' # Optional, will assume , for csv and \t otherwises
    header: 0 # Optional, 0 indicates first row is header, None indicates no header
    columns: ['column1', 'column2', 'column3'] # Optional, if not provided all columns will be used
```

Make sure to add that to your relevant section in your config (can be multiple times if you're working with different sheets or different columns), then call the function on this object and it'll either mention somethings wrong or return a pandas dataframe with the columns of interest.

This is an example of a common sample_sheet we work with. We will ingest the hash at the beginning so it doesn't affect column naming. Extra empty rows at the end are also stripped.
```tsv
#sample_id	file_path	metadata1	metadata2
Sample1	/path/to/sample1.fasta	value1	option1
Sample2	/path/to/sample2.fasta	value2	option2
Sample3	/path/to/sample3.fasta	value3	option1
Sample4	/path/to/sample4.fasta	value1	option2
Sample5	/path/to/sample5.fasta	value2	option1
```

In [ ]:
# | export
import pandas as pd


def get_samplesheet(sample_sheet_config: dict) -> pd.DataFrame:
    # Load the sample sheet into a pandas dataframe
    # If columns is not None then it will only load those columns
    # If the sample sheet is a csv then it will load it as a csv, otherwise it will assume it's a tsv

    # Expected sample_sheet_config:
    # sample_sheet:
    #   path: path/to/sample_sheet.tsv
    #   delimiter: '\t' # Optional, will assume , for csv and \t otherwises
    #   header: 0 # Optional, 0 indicates first row is header, None indicates no header
    #   columns: ['column1', 'column2', 'column3'] # Optional, if not provided all columns will be used

    # Example sample sheet:
    # #sample_id	file_path	metadata1	metadata2
    # Sample1	/path/to/sample1.fasta	value1	option1
    # Sample2	/path/to/sample2.fasta	value2	option2
    # Sample3	/path/to/sample3.fasta	value3	option1
    # Sample4	/path/to/sample4.fasta	value1	option2
    # Sample5	/path/to/sample5.fasta	value2	option1

    # This function should also handle ensuring the sample sheet is in the correct format, such as ensuring the columns are correct and that the sample names are unique.
    if not os.path.isfile(sample_sheet_config["path"]):
        raise FileNotFoundError(f"File {sample_sheet_config['path']} does not exist")
    if "delimiter" in sample_sheet_config:
        delimiter = sample_sheet_config["delimiter"]
    else:
        # do a best guess based on file extension
        delimiter = "," if sample_sheet_config["path"].endswith(".csv") else "\t"
    header = 0
    # if "header" in sample_sheet_config:
    #     header = sample_sheet_config["header"]
    # else:
    #     # check if the first line starts with a #, if so lets assume it's a header otherwise assume there isn't one
    #     with open(sample_sheet_config["path"], "r") as f:
    #         first_line = f.readline()
    #         header = 0 if first_line.startswith("#") else None
    if "columns" in sample_sheet_config:
        columns = sample_sheet_config[
            "columns"
        ]  # note the # for the first item needs to be stripped to compare to the columns
    else:
        columns = None  # implies all columns
    try:
        # note when we have a header the first column may begin with a #, so we need to remove it
        df = pd.read_csv(
            sample_sheet_config["path"],
            delimiter=delimiter,
            header=header,
            comment=None,
        )
    except Exception as e:
        print(
            "Error: Could not load sample sheet into dataframe, you have a problem with your sample sheet or the configuration."
        )
        raise e

    # Check the first header has a # in it, if so remove it for only that item
    if df.columns[0].startswith("#"):
        df.columns = [col.lstrip("#") for col in df.columns]
    # Ensure the sample sheet has the correct columns
    if columns is not None and not all([col in df.columns for col in columns]):
        raise ValueError("Error: Sample sheet does not have the correct columns")
    # also drop columns which are not needed
    if columns is not None:
        df = df[columns]

    # Clean the df of any extra rows that can be caused by empty lines in the sample sheet
    df = df.dropna(how="all")
    return df

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the associated python package
import nbdev

nbdev.nbdev_export()